# Jupyter notebook for anomaly 3d models

# Pip install libraries

In [ ]:
%pip install trame
%pip install vtk
%pip install fiona
%pip install gemgis
%pip install rasterio
%pip install pyvista
%pip install trimesh
%pip install scipy
%pip install pyglet
%pip install ipywidgets
%pip install ipyleaflet
%pip install ipython
%pip install plotly
%pip install auscopecat


In [ ]:
import re
import numpy as np
import plotly.graph_objects as go

# Prevent exit crashes
class StopExecutionError(Exception):
    def _render_traceback_(self):
        return []

# Read_ts function to read the GoCAD file.

In [ ]:
#TS parsing code is a  modified version of https://github.com/RichardScottOZ/GOCAD_TS-Surface-Reader/blob/main/GOCAD-Reader-example.ipynb
#Thanks for Richard Scott's contribution.
def read_ts(tsfile):
    vrtx = []
    trgl = []
    #split atom line by by space, make vertex at that point in VRTX list
    #go back and get atomlist[1]  VRTXLIST[atomlist[1]-1]

    for line in tsfile:
        if 'VRTX' in line:
            l_input = re.split(r'[\s]+', line)
            temp = np.array(l_input[2:5])
            #temp = np.array(l_input[2:6])
            vrtx.append(temp.astype(float))

        if 'ATOM' in line:
            l_input = re.split(r'[\s]+', line)
            #vertex_id = l_input[1]
            vertex_id_atom = l_input[2]
            #print(len(vrtx), vertex_id, vertex_id_atom)
            vrtx.append(vrtx[ int(vertex_id_atom) -1])

        if 'TRGL' in line:
            l_input = re.split(r'[\s]+', line)
            temp = np.array(l_input[1:4])
            trgl.append(temp.astype(int))

    vrtx = np.asarray(vrtx)
    trgl = np.asarray(trgl)

    crs_dict = {"NAME":None,"AXIS_NAME":None,"AXIS_UNIT":None,"ZPOSITIVE":None}
    check_dist = {"SURFACE":tsfile[-1],"NAME":None,"COORD":0,"TFACE":0,"PVRTX":0,"VRTX":0,"TRGL":0,"ATOM":0,"CRS":crs_dict, "COLOR":None}

    for line in tsfile:
        ##HEADER
        if 'color:' in line:
            strcolor = line.split(':')
            tricolor = re.split(r'[\s]+', strcolor[1])
            tricolor.pop()
            tricolor[0] = float(tricolor[0]) #rgb 0-1 colors R
            tricolor[1] = float(tricolor[1]) #rgb 0-1 colors G
            tricolor[2] = float(tricolor[2]) #rgb 0-1 colors B
            check_dist["COLOR"] = tricolor
        if 'name:' in line:
            strname = line.split(':')
            usename = strname[1].replace("\n",'')
            check_dist["NAME"] = usename
        if 'NAME ' in line and "AXIS" not in line:
            strname = re.split(r'[\s]+', line)
            crs = strname[1].replace("\n",'')
        if 'AXIS_NAME' in line:
            axis_name = re.split(r'[\s]+', line)
            axis_name.pop()
        if 'AXIS_UNIT' in line:
            axis_unit = re.split(r'[\s]+', line)
            axis_unit.pop()
        if 'ZPOSITIVE' in line:
            zpositive = re.split(r'[\s]+', line)[1].replace("\n",'')
        if 'END_ORIGINAL_COORDINATE' in line:
            crs_dict["NAME"] = crs
            crs_dict["AXIS_NAME"] = axis_name
            crs_dict["AXIS_UNIT"] = axis_unit
            crs_dict["ZPOSITIVE"] = zpositive

        ####Data
        if 'TFACE' in line:
            check_dist["TFACE"] += 1
        if 'COORDINATE_SYSTEM' in line:
            check_dist["COORD"] += 1
        if 'ATOM' in line:
            check_dist["ATOM"] += 1
        if 'PVRTX' in line:
            check_dist["PVRTX"] += 1
        if 'VRTX' in line:
            check_dist["VRTX"] += 1
        if 'TRGL' in line:
            check_dist["TRGL"] += 1

    return vrtx, trgl, check_dist

# User input for search
# User click to select a borehole for 3D model

In [ ]:
#Get bbox for search
import ipywidgets as widgets
from ipyleaflet import CircleMarker, GeomanDrawControl, Map, Rectangle, basemaps
from IPython.display import Markdown, display

from auscopecat.api import search_cql

AUSTRALIA_BBOX = {'north': -10.6681, 'east': 153.5694, 'south': -43.6345, 'west': 113.3389}
#Get anomalyName for search
anomaly_name_text = widgets.Text(value='Pilbara', placeholder='the anomaly Name to search.', description='Name:',layout=widgets.Layout(width='200px'))

display(Markdown('## 1.Input a Name'))
display(Markdown('## 2.Draw a BBox or Click button of Select Australia'))
display(Markdown('## 3.Click button of Search Anomaly'))
display(Markdown('## 4.Click a borehole dot to select a 3D model'))
australia_btn = widgets.Button(description = 'Select Australia', button_style = '', tooltip = 'Reset bounds to Australia', icon = '')
search_btn = widgets.Button(description = 'Search Anomaly', button_style = '', tooltip = 'Search Anomaly', icon = '')
north_field = widgets.BoundedFloatText(description = 'North', value = AUSTRALIA_BBOX['north'], min = -90.0, max = 90.0, style = {'font_size': '170%' }, layout=widgets.Layout(width='200px'))
west_field = widgets.BoundedFloatText(description = 'West', value = AUSTRALIA_BBOX['west'], min = -180.0, max = 180.0, style = {'font_size': '170%' }, layout=widgets.Layout(width='200px'))
south_field = widgets.BoundedFloatText(description = 'South', value = AUSTRALIA_BBOX['south'], min = -90.0, max = 90.0, style = {'font_size': '170%'}, layout=widgets.Layout(width='200px'))
east_field = widgets.BoundedFloatText(description = 'East', value = AUSTRALIA_BBOX['east'], min = -180.0, max = 180.0, style = {'font_size': '170%'}, layout=widgets.Layout(width='200px'))
input_box_layout = widgets.Layout(display = 'flex', flex_flow = 'row', align_items = 'stretch')
input_box = widgets.Box(children=[north_field, west_field, south_field, east_field, australia_btn], layout = input_box_layout)

display(Markdown('\n#### User Input Box'))
display(anomaly_name_text)
display(input_box)
display(search_btn)

default_layout = widgets.Layout(width='1024px', height='768px')
m = Map(basemap = basemaps.Esri.WorldStreetMap, center = (-29.6, 133.0), zoom = 4, layout = default_layout)
df = None
def reset_bounds(b):
    north_field.value = AUSTRALIA_BBOX['north']
    south_field.value = AUSTRALIA_BBOX['south']
    west_field.value = AUSTRALIA_BBOX['west']
    east_field.value = AUSTRALIA_BBOX['east']
    rectangle = Rectangle(bounds=((AUSTRALIA_BBOX['south'],AUSTRALIA_BBOX['west']),(AUSTRALIA_BBOX['north'],AUSTRALIA_BBOX['east'])))
    # Add the rectangle to the map
    m.add(rectangle)

def search_anomaly(b):
    global df
    #Searching
    bbox = f'{float(west_field.value):.2f}, {float(south_field.value):.2f}, {float(east_field.value):.2f}, {float(north_field.value):.2f}'
    name = f'{anomaly_name_text.value}' #'Pilbara'
    cql_filter = ''
    if name :
        if cql_filter.strip() != "":
            cql_filter +=  ' AND '
        cql_filter += f'AnomalyName like \'%{name}%\''

    if bbox :
        if cql_filter.strip() != "":
            cql_filter +=  ' AND '
        cql_filter += f'BBOX(CentreLocation,{bbox})'

    print(f'searching for {cql_filter}')
    # cql_filter = f'AnomalyName like \'%{name}%\''
    url = 'https://remanentanomalies.csiro.au/geoserver/ows'
    params = {
                'service': 'WFS',
                'version': '1.1.0',
                'request': 'GetFeature',
                'typename': 'RemAnom:Anomaly',
                'outputFormat': 'csv',
                'srsname': 'EPSG:4326',
                'CQL_FILTER': cql_filter,
                'maxFeatures': str(10000)
                }
    try:
        df = search_cql(url, params)
    except Exception as e:
        print(f'Error querying data: {e}', 500)
    print(df.shape)
    df.head(10)

    #clean all the markerLayers first
    for layer in m.layers:
        if isinstance(layer, CircleMarker):
            m.remove_layer(layer)
    #add circle-dot for anomalies boreholes:
    for index, row in enumerate(df.itertuples()):
        # if index > 10:
        #     break
        point = row[29]
        found = re.search(r'\((.+?)\)', point)
        if not found:
            continue
        cx, cy = found.group(1).split(' ')
        selected_anomaly = f'{row[1]}@{row[7]}'
        html = widgets.HTML()
        html.value = selected_anomaly
        marker = CircleMarker(name = selected_anomaly, location = (cy,cx),radius = 5, color = 'green', fillcolor = 'green')
        marker.on_click(create_popup_click(val1 = selected_anomaly))
        m.add_layer(marker)
        marker.popup = html

australia_btn.on_click(reset_bounds)
search_btn.on_click(search_anomaly)

draw_control = GeomanDrawControl()
draw_control.circlemarker = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.rectangle = {'shapeOptions': {'fillColor': '#fca45d','color': '#fca45d','fillOpacity': 0.7}}

def handle_draw(target, action, geo_json):
    if action == 'create' or action == 'drag':
        coords = geo_json[0]['geometry']['coordinates'][0]
        if len(coords) == 5:
            west_field.value = str(coords[0][0])
            south_field.value = str(coords[0][1])
            east_field.value = str(coords[2][0])
            north_field.value = str(coords[2][1])
            # TODO: Enable Rectangle control (possible?)
    elif action == 'remove':
        # TODO: Disable Rectangle control (possible?)
        pass
selected_anomaly =''

def create_popup_click(val1):
    def popup_click(event, type, coordinates, val1 = val1):
        global selected_anomaly
        selected_anomaly = str(val1)
    return popup_click
reset_bounds(None)
draw_control.on_draw(handle_draw)
m.add(draw_control)
m

In [ ]:
#show the search result in table.df
if df is not None:
    print(df.shape)
else:
    alert_msg = '<div class="alert alert-block alert-danger"><strong>Alert!!!</strong> Could not find any anomaly. Please change a name or bbox ,then search again!</div>'
    display(Markdown(alert_msg))
    raise StopExecutionError
df.head(10)

In [ ]:
import os
import urllib
from io import BytesIO
from zipfile import BadZipFile, ZipFile

import matplotlib.pyplot as plt
import requests
from IPython.display import Markdown, display
from PIL import Image

try:
    anomaly_id = selected_anomaly.split('@')[0].split('.')[1]
except Exception as ex:
    print(str(ex))
    print('Could not get anomalyID. Please click the dot to select one first!')
    raise StopExecutionError
print(anomaly_id)
anomaly_jpeg_url = f'https://remanentanomalies.csiro.au/getJpeg.ashx?anomalyId={anomaly_id}'
print (anomaly_jpeg_url)
anomaly_3d_model_url = f'https://remanentanomalies.csiro.au/getAllModelsForAnomaly.ashx?anomalyid={anomaly_id}'
print (anomaly_3d_model_url)

#download anomaly_3d_model_url
try:
    z = ZipFile(BytesIO(urllib.request.urlopen(anomaly_3d_model_url).read()))
except BadZipFile:
    alert_msg = f'<div class="alert alert-block alert-danger"><strong>Alert!!!</strong> Could not find the 3D TS model for this anomaly.{anomaly_id}. Please go back to click another borehole dot and try again.</div>'
    display(Markdown(alert_msg))
    raise StopExecutionError

anomaly_model_filename:str = ''
for name in z.namelist():
    if '.zip' not in name:
        continue
    z2 = ZipFile(BytesIO(z.read(name)))
    for name2 in z2.namelist():
        if '.ts' not in name2:
            continue
        ts_file = z2.read(name2)
        anomaly_model_filename = os.path.basename(name2)
        with open(f'{anomaly_model_filename}',mode='wb') as f:
            f.write(ts_file)

#download anomaly_jpeg
anomaly_jpeg = Image.open(requests.get(anomaly_jpeg_url, stream=True).raw)
plt.imshow(anomaly_jpeg)
anomaly_jpeg.save(f'magnetic-anomaly-{anomaly_id}.jpg')

In [ ]:
#Visualize the Anomaly 3D by Plotly

def float_to_hex_color(color_float):
    r_float, g_float, b_float, a_float = color_float
    #Converts float color components (0.0-1.0) to a hex color string (#RRGGBB or #RRGGBBAA).
    def to_hex_component(value):
        scaled_value = int(value * 255)
        return f"{scaled_value:02x}" # Format as two-digit hex, padding with zero if needed

    hex_r = to_hex_component(r_float)
    hex_g = to_hex_component(g_float)
    hex_b = to_hex_component(b_float)

    if a_float == 1.0:
        return f"#{hex_r}{hex_g}{hex_b}"
    else:
        hex_a = to_hex_component(a_float)
        return f"#{hex_r}{hex_g}{hex_b}{hex_a}"

# example: https://remanentanomalies.csiro.au/getAllModelsForAnomaly.ashx?anomalyid=261
# anomaly_model_filename = r'E:\work\EMAG2_4_model_4\EMAG2_anomaly_4_model_4.ts'
with open(f'{anomaly_model_filename}',mode='rt') as f:
    ts_file = f.read()
    v, t, meta = read_ts(ts_file.splitlines())
    tf = t-1
    colorFloat = meta["COLOR"]
    if len(colorFloat) <4:
        colorFloat.append(1) #alpha
    color  = float_to_hex_color(colorFloat)

    fig = go.Figure(data=[go.Mesh3d(x=v[:,0], y=v[:,1], z=v[:,2], i=tf[:,0], j=tf[:,1], k=tf[:,2], opacity=0.5, color=color)])
    fig.update_layout(title='3D Anomaly')
    fig.show()